In [1]:
import torch
import numpy as np
import pandas as pd
from torch_geometric.utils import remove_self_loops
from torch_sparse import coalesce
from random import sample, seed
from torch_geometric.data import Data
from torch_geometric_node2vec import Node2Vec

In [2]:
def load_embedding(embFile,featFile = None):
    xDict = {}
    yDict = {}
    
    rename_class = {}
    cnt_class = 0
    with open(featFile, 'r') as f:
        for line in f:
            s = line.split()
            id = s[0]
            emb = [float(x) for x in s[1:-1]]
            xDict[id] = emb
            if s[-1] not in rename_class:
                rename_class[s[-1]] = cnt_class
                cnt_class += 1
            yDict[id] = rename_class[s[-1]]

    rename_id={}
    cnt = 0
    for k in xDict.keys():
        rename_id[cnt] = k
        cnt += 1
            
    # Read embedding
    err = 0
    with open(embFile, 'r') as f:
        skip = True
        for line in f:
            s = line.split()
            if skip and len(s)==2:
                skip = False
                continue
            skip = False
            id = s[0]
            emb = [float(x) for x in s[1:]]
            if id in xDict:
                xDict[id] = xDict[id] + emb
            else:
                err +=1

    print(err)

    
    x = []
    target = []
    err = 0
    for i in range(cnt):
#         if (len(xDict[rename_id[i]])!=1561):
#             err+=1
#             continue
        x.append(np.array(xDict[rename_id[i]]))
        target.append(yDict[rename_id[i]])
    print(err)

    
    
    x = torch.tensor(np.array(x),dtype=torch.float)
    y = torch.tensor(np.array(target),dtype=torch.long)
    
    num_classes = len(set(target))
    df = pd.DataFrame(target)
    df.columns = ['target']
    train = []
    
    n = len(target)
    
    seed(1)
    for i in range(num_classes):
        train = train + sample(df[df['target']==i].index.values.tolist() ,20)
    rest = [i for i in range(n) if i not in train]
    val = rest[:500]
    test = rest[500:1500]
    train_ind = [1 if i in train else 0 for i in range(n)]
    val_ind = [1 if i in val else 0 for i in range(n)]
    test_ind = [1 if i in test else 0 for i in range(n)]

    data = Data(x=x, y=y)
    data.test_mask = torch.tensor(test_ind,dtype=torch.uint8)
    data.train_mask = torch.tensor(train_ind,dtype=torch.uint8)
    data.val_mask = torch.tensor(val_ind,dtype=torch.uint8)

    
    return data
    




In [12]:
import torch
from torch_geometric.data import InMemoryDataset, Data
import os.path as osp
from citation_loader import read_network
from shutil import copyfile

class EmbeddingData(InMemoryDataset):
    
    embUrl = '../data/embedding'
    featsUrl = '../data/raw'

    def __init__(self, root, name, method, directed,reverse=False): #, transform=None, pre_transform=None):
        self.name = name
        self.method = method
        self.directed = directed
        self.reverse = reverse
        
        super(EmbeddingData, self).__init__(root, None, None)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return ['{}.{}.{}.emb'.format(self.name,self.method,'reversed' if self.reverse else 
                                      ('directed' if self.directed else 'undirected')),
                '{}.content'.format(self.name)]

    @property
    def processed_file_names(self):
        return 'data.pt'

    def download(self):
        copyfile('{}/{}'.format(self.embUrl, self.raw_file_names[0]),'{}/{}'.format(self.raw_dir, self.raw_file_names[0]))
        copyfile('{}/{}/{}'.format(self.featsUrl, self.name, self.raw_file_names[1]),'{}/{}'.format(self.raw_dir, self.raw_file_names[1]))

    def process(self):
        emb_path = osp.join(self.raw_dir,self.raw_file_names[0])
        features_path = osp.join(self.raw_dir,self.raw_file_names[1])
        data = load_embedding(emb_path,features_path)
        torch.save(self.collate([data]), self.processed_paths[0])
        
    def __repr__(self):
        return '{}()'.format(self.name)

In [4]:
class FeatureNNClassifier(torch.nn.Module):
    def __init__(self, n_in, n_h, n_out):
        super(FeatureNNClassifier, self).__init__()
        self.linear1 = torch.nn.Linear(n_in, n_h)
        self.linear2 = torch.nn.Linear(n_h, n_out)

    def forward(self, x):
        x = self.linear1(x)
        x = F.relu(x)
        x = self.linear2(x)
        x = F.log_softmax(x,dim=1)
        return x


In [5]:
import torch.nn.functional as F

In [6]:
# Define a baseline
def eval_baseline_run(dataset,data,n_h=128,n_out=7):
    n_in = dataset.num_features
#     n_out = dataset.num_classes
    baseline = FeatureNNClassifier(n_in,n_h,n_out)
    device = 'cpu'
    data = dataset[0].x.to(device)
    optimizer = torch.optim.Adam(baseline.parameters(),lr=0.01,weight_decay=5e-4)
    baseline.train() # to enter training phase
    for epoch in range(200):
        optimizer.zero_grad() # saw this a lot in the beginning, maybe resetting gradients (not to accumulate)
        out = baseline(data) # this calls the forward method apparently
        loss = F.nll_loss(out[dataset[0].train_mask],dataset[0].y[dataset[0].train_mask]) # nice indexing, easy and short
        loss.backward() # magic: real back propagation step, takes care of the gradients and stuff
        optimizer.step() # maybe updates the params to be optimized
    baseline.eval() # enter eval phase
    _,pred = baseline(data).max(dim=1) # take prediction out of softmax
    correct = float(pred[dataset[0].test_mask].eq(dataset[0].y[dataset[0].test_mask]).sum().item())
    acc = correct / dataset[0].test_mask.sum().item()
    return acc
def eval_baseline(dataset,n_h=128,runs=100):
    accs = []
    for i in range(runs):
        accs.append(eval_baseline_run(dataset,n_h))
    return sum(accs)/len(accs),accs

In [7]:
def test_n2v(data):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    z = data.x
    z = z.to(device)
    n2v = Node2Vec(num_nodes=data.num_nodes,
              embedding_dim=data.x.shape[1],
              walk_length=80,
              context_size=10,
              walks_per_node=10,
              p=1,q=1)
    n2v = n2v.to(device)
    acc = n2v.test(z[data.train_mask], data.y[data.train_mask],
                     z[data.test_mask], data.y[data.test_mask], max_iter=150)
    print("{:.2f}".format(acc*100))
#     return acc

In [11]:
from network_split import NetworkSplitShcur

In [15]:
def eval_n2v(data,num_splits=100):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    z = data.x
    z = z.to(device)
    n2v = Node2Vec(num_nodes=data.num_nodes,
              embedding_dim=data.x.shape[1],
              walk_length=80,
              context_size=10,
              walks_per_node=10,
              p=1,q=1)
    accs = []
    for i in range(num_splits):
        split = NetworkSplitShcur(data,early_examples_per_class=0,split_seed=i)
        n2v = n2v.to(device)
        acc = n2v.test(z[split.train_mask], data.y[split.train_mask],
                         z[split.val_mask], data.y[split.val_mask], max_iter=150)
#         print("{:.2f}".format(acc*100))
        accs.append(acc)
    return np.mean(accs)

### Cora
Baseline Accuracy: 0.5581

#### node2vec

Param search

In [16]:
emb = EmbeddingData('/tmp/n2vUndp0.25EmbCora','cora','node2vec_rw.p0.25.q1.75',directed=False)
eval_n2v(emb[0])

72.38
64.76
75.71
70.95


KeyboardInterrupt: 

In [120]:
emb = EmbeddingData('/tmp/n2vUndp0.75EmbCora','cora','node2vec_rw.p0.75.q1.25',directed=False)
eval_n2v(emb[0])

68.40


In [121]:
emb = EmbeddingData('/tmp/n2vUndp1.25EmbCora','cora','node2vec_rw.p1.25.q0.75',directed=False)
eval_n2v(emb[0])

68.60


In [122]:
emb = EmbeddingData('/tmp/n2vUndp1.75EmbCora','cora','node2vec_rw.p1.75.q0.25',directed=False)
eval_n2v(emb[0])

68.60


In [123]:
emb = EmbeddingData('/tmp/n2vUndEmbCora','cora','node2vec_rw',directed=False)
eval_n2v(emb[0])

67.80


p = 0.25, q = 1.75

In [124]:
emb = EmbeddingData('/tmp/n2vUndp0.25EmbCora','cora','node2vec_rw.p0.25.q1.75',directed=False)
test_n2v(emb[0])

72.30


In [126]:
emb = EmbeddingData('/tmp/n2vDirp0.25EmbCora','cora','node2vec_rw.p0.25.q1.75',directed=True)
test_n2v(emb[0])

56.40


In [202]:
emb = EmbeddingData('/tmp/n2vRevEmbCora','cora','node2vec_rw',directed=True,reverse=True)
eval_baseline_run(emb,None,n_h=16,n_out=emb.num_classes)

0.558

#### HOPE

param search

In [238]:
emb = EmbeddingData('/tmp/Hopeadamic-adarUndEmbCora','cora','hope_adamic-adar',directed=False)
eval_n2v(emb[0])

Processing...
0
0
Done!
54.80


In [239]:
emb = EmbeddingData('/tmp/Hopecommon-neighborsUndEmbCora','cora','hope_common-neighbors',directed=False)
eval_n2v(emb[0])

Processing...
0
0
Done!
60.60


In [240]:
emb = EmbeddingData('/tmp/Hopekatz_0.1UndEmbCora','cora','hope_katz_0.1',directed=False)
eval_n2v(emb[0])

Processing...
0
0
Done!
55.40


In [241]:
emb = EmbeddingData('/tmp/Hopekatz_0.01UndEmbCora','cora','hope_katz_0.01',directed=False)
eval_n2v(emb[0])

Processing...
0
0
Done!
54.00


In [242]:
emb = EmbeddingData('/tmp/Hopekatz_0.9UndEmbCora','cora','hope_katz_0.9',directed=False)
eval_n2v(emb[0])

Processing...
0
0
Done!
50.00


In [243]:
emb = EmbeddingData('/tmp/Hoperpr_0.5UndEmbCora','cora','hope_rpr_0.5',directed=False)
eval_n2v(emb[0])

Processing...
0
0
Done!
54.00


In [244]:
emb = EmbeddingData('/tmp/Hoperpr_0.85UndEmbCora','cora','hope_rpr_0.85',directed=False)
eval_n2v(emb[0])

Processing...
0
0
Done!
54.00


In [245]:
emb = EmbeddingData('/tmp/Hoperpr_0.99UndEmbCora','cora','hope_rpr_0.99',directed=False)
eval_n2v(emb[0])

Processing...
0
0
Done!
54.00


In [246]:
emb = EmbeddingData('/tmp/Hopecommon-neighborsUndEmbCora','cora','hope_common-neighbors',directed=False)
test_n2v(emb[0])

64.00


In [210]:
emb = EmbeddingData('/tmp/Hopecommon-neighborsDirEmbCora','cora','hope.common-neighbors',directed=True)
test_n2v(emb[0])
emb = EmbeddingData('/tmp/Hopecommon-neighborsRevEmbCora','cora','hope.common-neighbors',directed=True,reverse=True)
test_n2v(emb[0])

59.50
59.50


#### LINE(1)

Param search

In [60]:
emb = EmbeddingData('/tmp/Line1.5UndEmbCora','cora','line1.5',directed=False)
eval_n2v(emb[0])

65.60


In [63]:
emb = EmbeddingData('/tmp/Line1.10UndEmbCora','cora','line1.10',directed=False)
eval_n2v(emb[0])

63.40


In [64]:
emb = EmbeddingData('/tmp/Line1.15UndEmbCora','cora','line1.15',directed=False)
eval_n2v(emb[0])

61.60


In [65]:
emb = EmbeddingData('/tmp/Line1UndEmbCora','cora','line1',directed=False)
eval_n2v(emb[0])

61.80


We set NS=5

In [173]:
emb = EmbeddingData('/tmp/Line1.5UndEmbCora','cora','line1.5',directed=False)
test_n2v(emb[0])

67.00


In [207]:
emb = EmbeddingData('/tmp/Line1DirEmbCora','cora','line1',directed=True)
test_n2v(emb[0])

57.60


In [208]:
emb = EmbeddingData('/tmp/Line1RevEmbCora','cora','line1',directed=True,reverse=True)
test_n2v(emb[0])

58.10


#### LINE(2)

Param search

In [77]:
emb = EmbeddingData('/tmp/Line2.5UndEmbCora','cora','line2.5',directed=False)
eval_n2v(emb[0])

64.400000


In [78]:
emb = EmbeddingData('/tmp/Line2.10UndEmbCora','cora','line2.10',directed=False)
eval_n2v(emb[0])

64.400000


In [72]:
emb = EmbeddingData('/tmp/Line2.15UndEmbCora','cora','line2.15',directed=False)
eval_n2v(emb[0])

61.20


In [66]:
emb = EmbeddingData('/tmp/Line2UndEmbCora','cora','line2',directed=False)
eval_n2v(emb[0])

61.60


We set NS=5

In [80]:
emb = EmbeddingData('/tmp/Line2.5UndEmbCora','cora','line2.5',directed=False)
test_n2v(emb[0])

67.00


In [83]:
emb = EmbeddingData('/tmp/Line2.5DirEmbCora','cora','line2.5',directed=True)
test_n2v(emb[0])

57.20


In [211]:
emb = EmbeddingData('/tmp/Line2.5RevEmbCora','cora','line2.5',directed=True,reverse=True)
test_n2v(emb[0])

58.70


#### NERD

Param search

In [46]:
# WL=3
emb = EmbeddingData('/tmp/NerdDirHubEmbCora','cora','nerd.hub',directed=True)
eval_n2v(emb[0])
emb = EmbeddingData('/tmp/NerdDirHubEmbCora','cora','nerd.hub',directed=True)
test_n2v(emb[0])

57.60
59.60


In [45]:
# WL=5
emb = EmbeddingData('/tmp/NerdDirHub5EmbCora','cora','nerd.hub.5',directed=True)
eval_n2v(emb[0])
emb = EmbeddingData('/tmp/NerdDirAut5EmbCora','cora','nerd.aut.5',directed=True)
eval_n2v(emb[0])

61.00
62.80


In [41]:
# WL=10
emb = EmbeddingData('/tmp/NerdDirHub10EmbCora','cora','nerd.hub.10',directed=True)
eval_n2v(emb[0])
emb = EmbeddingData('/tmp/NerdDirAut10EmbCora','cora','nerd.aut.10',directed=True)
eval_n2v(emb[0])

59.00
66.60


In [48]:
# WL=15
emb = EmbeddingData('/tmp/NerdDirHub15EmbCora','cora','nerd.hub.15',directed=True)
eval_n2v(emb[0])
emb = EmbeddingData('/tmp/NerdDirAut15EmbCora','cora','nerd.aut.15',directed=True)
eval_n2v(emb[0])

58.60
67.00


In [51]:
# WL=20
emb = EmbeddingData('/tmp/NerdDirHub20EmbCora','cora','nerd.hub.20',directed=True)
eval_n2v(emb[0])
emb = EmbeddingData('/tmp/NerdDirAut20EmbCora','cora','nerd.aut.20',directed=True)
eval_n2v(emb[0])

57.40
66.20


Seems WL=15 is the solution

In [53]:
# NS=15
emb = EmbeddingData('/tmp/NerdDirHub15.15EmbCora','cora','nerd.hub.15.15',directed=True)
eval_n2v(emb[0])
emb = EmbeddingData('/tmp/NerdDirAut15.15EmbCora','cora','nerd.aut.15.15',directed=True)
eval_n2v(emb[0])

59.20
67.20


In [55]:
# NS=10
emb = EmbeddingData('/tmp/NerdDirHub15.10EmbCora','cora','nerd.hub.15.10',directed=True)
eval_n2v(emb[0])
emb = EmbeddingData('/tmp/NerdDirAut15.10EmbCora','cora','nerd.aut.15.10',directed=True)
eval_n2v(emb[0])

58.00
64.40


In [57]:
# NS=5
emb = EmbeddingData('/tmp/NerdDirHub15.5EmbCora','cora','nerd.hub.15.5',directed=True)
eval_n2v(emb[0])
emb = EmbeddingData('/tmp/NerdDirAut15.5EmbCora','cora','nerd.aut.15.5',directed=True)
eval_n2v(emb[0])

58.80
66.00


Seems NS=15 is the solution

In [58]:
emb = EmbeddingData('/tmp/NerdDirHub15.15EmbCora','cora','nerd.hub.15.15',directed=True)
test_n2v(emb[0])
emb = EmbeddingData('/tmp/NerdDirAut15.15EmbCora','cora','nerd.aut.15.15',directed=True)
test_n2v(emb[0])

61.20
67.90


In [146]:
emb = EmbeddingData('/tmp/NerdUndHub15.15EmbCora','cora','nerd.hub.15.15',directed=False)
test_n2v(emb[0])
emb = EmbeddingData('/tmp/NerdUndAut15.15EmbCora','cora','nerd.aut.15.15',directed=False)
test_n2v(emb[0])

75.70
75.30


### CiteSeer
Baseline Accuracy: 0.6123

#### node2vec

In [129]:
emb = EmbeddingData('/tmp/n2vUndp0.25EmbCiteseer','citeseer','node2vec_rw.p0.25.q1.75',directed=False)
test_n2v(emb[0])

65.30


In [130]:
emb = EmbeddingData('/tmp/n2vDirp0.25EmbCiteseer','citeseer','node2vec_rw.p0.25.q1.75',directed=True)
test_n2v(emb[0])

60.60


In [217]:
emb = EmbeddingData('/tmp/n2vRevEmbCiteseer','citeseer','node2vec_rw',directed=True,reverse=True)
test_n2v(emb[0])

61.70


#### HOPE

In [247]:
emb = EmbeddingData('/tmp/Hopecommon-neighbors_UndEmbCiteseer','citeseer','hope_common-neighbors',directed=False)
test_n2v(emb[0])

Processing...
15
0
Done!
61.40


In [208]:
emb = EmbeddingData('/tmp/Hopecommon-neighborsDirEmbCiteseer','citeseer','hope.common-neighbors',directed=True)
test_n2v(emb[0])
emb = EmbeddingData('/tmp/Hopecommon-neighborsRevEmbCiteseer','citeseer','hope.common-neighbors',directed=True,reverse=True)
test_n2v(emb[0])

61.20
61.20


LINE did not work for citeseer in directed and reversed cases

#### LINE(1)

In [159]:
emb = EmbeddingData('/tmp/Line1.5UndEmbCiteseer','citeseer','line1.5',directed=False)
test_n2v(emb[0])

63.90


#### LINE(2)

In [163]:
emb = EmbeddingData('/tmp/Line2.5UndEmbCiteseer','citeseer','line2.5',directed=False)
test_n2v(emb[0])

Processing...
15
0
Done!
66.40


In [166]:
emb = EmbeddingData('/tmp/Line2.5DirEmbCiteseer','citeseer','line2.5',directed=True)
test_n2v(emb[0])

Processing...
15
0
Done!
56.90


In [167]:
emb = EmbeddingData('/tmp/Line2.5RevEmbCiteseer','citeseer','line2.5',directed=True,reverse=True)
test_n2v(emb[0])

Processing...
15
0
Done!
62.30


#### NERD

In [258]:
emb = EmbeddingData('/tmp/NerdDirAutEmbCiteseer','citeseer','nerd.aut',directed=True)
test_n2v(emb[0])

62.10


In [259]:
emb = EmbeddingData('/tmp/NerdDirHubEmbCiteseer','citeseer','nerd.hub',directed=True)
test_n2v(emb[0])

57.40


In [260]:
emb = EmbeddingData('/tmp/NerdDirEmbCiteseer','citeseer','nerd',directed=True)
test_n2v(emb[0])

58.50


In [155]:
emb = EmbeddingData('/tmp/NerdDirHub15.15EmbCiteseer','citeseer','nerd.hub.15.15',directed=True)
test_n2v(emb[0])
emb = EmbeddingData('/tmp/NerdDirAut15.15EmbCiteseer','citeseer','nerd.aut.15.15',directed=True)
test_n2v(emb[0])

57.60
62.40


In [158]:
emb = EmbeddingData('/tmp/NerdUndHub15.15EmbCiteseer','citeseer','nerd.hub.15.15',directed=False)
test_n2v(emb[0])
emb = EmbeddingData('/tmp/NerdUndAut15.15EmbCiteseer','citeseer','nerd.aut.15.15',directed=False)
test_n2v(emb[0])

Processing...
15
0
Done!
65.10
Processing...
15
0
Done!
66.30


### PubMed
Baseline Accuracy: 0.6127

#### node2vec

In [138]:
emb = EmbeddingData('/tmp/n2vUndp0.25EmbPubMed','PubMed','node2vec_rw.p0.25.q1.75',directed=False)
test_n2v(emb[0])

55.50


In [139]:
emb = EmbeddingData('/tmp/n2vDirp0.25EmbPubMed','PubMed','node2vec_rw.p0.25.q1.75',directed=True)
test_n2v(emb[0])

54.30


In [225]:
emb = EmbeddingData('/tmp/n2vRevEmbPubMed','PubMed','node2vec_rw',directed=True,reverse=True)
test_n2v(emb[0])

39.40


/home/s-hussain/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


#### HOPE

In [248]:
emb = EmbeddingData('/tmp/Hopecommon-neighbors_UndEmbPubMed','PubMed','hope_common-neighbors',directed=False)
test_n2v(emb[0])

Processing...
0
0
Done!
70.00


In [204]:
emb = EmbeddingData('/tmp/Hopecommon-neighborsDirEmbPubMed','PubMed','hope.common-neighbors',directed=True)
test_n2v(emb[0])

68.20


In [205]:
emb = EmbeddingData('/tmp/Hopecommon-neighborsRevEmbPubMed','PubMed','hope.common-neighbors',directed=True,reverse=True)
test_n2v(emb[0])

68.20


In [226]:
emb = EmbeddingData('/tmp/HopeUndEmbPubMed','PubMed','hope_gsvd',directed=False)
test_n2v(emb[0])

67.30


In [227]:
emb = EmbeddingData('/tmp/HopeDirEmbPubMed','PubMed','hope_gsvd',directed=True)
test_n2v(emb[0])

67.30


In [228]:
emb = EmbeddingData('/tmp/HopeRevEmbPubMed','PubMed','hope_gsvd',directed=True,reverse=True)
test_n2v(emb[0])

66.90


#### LINE(1)

In [171]:
emb = EmbeddingData('/tmp/Line1.5UndEmbPubMed','PubMed','line1.5',directed=False)
test_n2v(emb[0])

72.50


In [230]:
emb = EmbeddingData('/tmp/Line1DirEmbPubMed','PubMed','line1',directed=True)
test_n2v(emb[0])

45.80


In [231]:
emb = EmbeddingData('/tmp/Line1RevEmbPubMed','PubMed','line1',directed=True,reverse=True)
test_n2v(emb[0])

49.60


#### LINE(2)

In [87]:
emb = EmbeddingData('/tmp/Line2.5UndEmbPubMed','PubMed','line2.5',directed=False)
test_n2v(emb[0])

65.20


In [90]:
emb = EmbeddingData('/tmp/Line2.5DirEmbPubMed','PubMed','line2.5',directed=True)
test_n2v(emb[0])

49.30


In [91]:
emb = EmbeddingData('/tmp/Line2.5RevEmbPubMed','PubMed','line2.5',directed=True,reverse=True)
test_n2v(emb[0])

58.20


#### NERD

In [235]:
emb = EmbeddingData('/tmp/NerdDirAutEmbPubMed','PubMed','nerd.aut',directed=True)
test_n2v(emb[0])

67.90


In [236]:
emb = EmbeddingData('/tmp/NerdDirHubEmbPubMed','PubMed','nerd.hub',directed=True)
test_n2v(emb[0])

52.50


In [237]:
emb = EmbeddingData('/tmp/NerdDirEmbPubMed','PubMed','nerd',directed=True)
test_n2v(emb[0])

67.90


In [148]:
emb = EmbeddingData('/tmp/NerdDirHub15.15EmbPubMed','PubMed','nerd.hub.15.15',directed=True)
test_n2v(emb[0])
emb = EmbeddingData('/tmp/NerdDirAut15.15EmbPubMed','PubMed','nerd.aut.15.15',directed=True)
test_n2v(emb[0])

53.60
71.50


In [151]:
emb = EmbeddingData('/tmp/NerdUndHub15.15EmbPubMed','PubMed','nerd.hub.15.15',directed=False)
test_n2v(emb[0])
emb = EmbeddingData('/tmp/NerdUndAut15.15EmbPubMed','PubMed','nerd.aut.15.15',directed=False)
test_n2v(emb[0])

72.70
72.80
